In [4]:
import dask_geopandas as dgpd
import dask.dataframe as dd
from shapely.geometry import Point, shape
import ast

# DATASET ENRICHMENT

The goal of this part of the task was to enrich the dataset with additional information about:
- weather (based on the time and location)
- Schools in vicinity (based on location)
- Events (based on time and location)
- Businesses in vicinity (based on location)


## BUSINESS DATA

First, we collect the business data from the followin source: 'https://data.cityofnewyork.us/Business/Issued-Licenses/w7w3-xahh/about_data'. The source contains various information about businesses in NYC, however, the data that we're interested in is mainly the name of the business and the location. Ideally, we would like to have the location information in the form of coordinates, however, a lot of entries in the dataset are missing that information. In order to get around that problem, we use the ZIP Code information to infer approximate location. We use a dataset obtained from 'https://download.geonames.org/export/zip/' to map various US ZIP codes to shape files, from which we can then calculate center points. The two datasets are then joined on the zipcodes and missing coordinates are filled with calculated approximations.

### ZIP Data

In [5]:
def load_zip_data(PATH):
    dtype = {
        1: 'int64',   # ZIP Code
        4: 'object',  # State
        9: 'float64', # lat
        10: 'float64' # lng
    }
    zip = dd.read_csv(PATH, sep='\t', header=None, dtype=dtype, usecols=[1, 4, 9, 10])
    zip.columns = ['ZIP Code', 'State', 'lat', 'lng']
    return zip

In [6]:
ZIP_PATH = 'data/US.txt'
zip_df = load_zip_data(ZIP_PATH)
# zip_df = zip_df.compute()
# print(zip_df.head())

### Business data

In [7]:
def load_business_data(PATH):
    bsns = dd.read_csv(
    PATH,
    usecols=['Business Name', 'ZIP Code', 'Latitude', 'Longitude'],
    dtype={'ZIP Code': 'str', # Some of the entries are not numeric, manual conversion needed
           'Business Name': 'object',
           'Longitude': 'float64',
           'Latitude': 'float64'}  # still useful to enforce type
)

    bsns['ZIP Code'] = dd.to_numeric(bsns['ZIP Code'], errors='coerce')
    bsns = bsns.dropna(subset=['ZIP Code'])
    bsns['ZIP Code'] = bsns['ZIP Code'].astype('int64') # This has to be enforced to ensure merge works
    return bsns





In [8]:
BUSINESS_PATH = 'data/businesses.csv'
bsns_gdf = load_business_data(BUSINESS_PATH)
# bsns_gdf = bsns_dgf.compute()
# print(bsns_gdf.head())


### Merge business and zip

In [9]:
def merge_business_zip(bsns, zip_ddf):
    bsns = bsns.merge(zip_ddf, how='left', on='ZIP Code')
    bsns['Latitude'] = bsns['Latitude'].fillna(bsns['lat'])
    bsns['Longitude'] = bsns['Longitude'].fillna(bsns['lng'])
    bsns = bsns.drop(columns=['lat', 'lng', 'State', 'ZIP Code'])
    bsns = bsns.dropna(subset=['Latitude', 'Longitude'])

    def make_point(lon, lat):
        return Point(lon, lat)


    # Apply the make_point function to create a 'geometry' column

    bsns = bsns.map_partitions(
        lambda df: df.assign(
            geometry=df.apply(lambda x: make_point(x['Longitude'], x['Latitude']), axis=1)
        ), meta={'Business Name': 'str', 'Latitude': 'float64', 'Longitude': 'float64', 'geometry': 'geometry'} # Dask needs to know the type of the new column
    )

    bsns = dgpd.from_dask_dataframe(bsns, geometry='geometry')
    bsns = bsns.set_crs("EPSG:4326").to_crs(3857)
    bsns = bsns.drop(columns=['Latitude', 'Longitude'])

    return bsns

In [10]:
bsns = merge_business_zip(bsns_gdf, zip_df)
bsns = bsns.compute()
print(bsns.head())

                 Business Name                          geometry
0             Denis Spedalieri  POINT (-8235727.623 4963332.949)
1  SANJAY'S VARIETY STORE INC.  POINT (-8211964.586 4969967.308)
2                 Gayla Hibner    POINT (-8237453.076 4971936.9)
3    FAMILY CARE REFERRAL, LLC   POINT (-8235658.16 4976635.563)
4                   Donna Hill  POINT (-8228124.502 4962687.224)


/home/sancho/miniconda3/envs/bigData/lib/python3.11/site-packages/dask/dataframe/dask_expr/_expr.py:1543: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return get_meta_library(args[0]).to_datetime(*args, **kwargs)
/home/sancho/miniconda3/envs/bigData/lib/python3.11/site-packages/dask/dataframe/dask_expr/_expr.py:1543: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return get_meta_library(args[0]).to_datetime(*args, **kwargs)
/home/sancho/miniconda3/envs/bigData/lib/python3.11/site-packages/dask/dataframe/dask_expr/_expr.py:1543: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  

## WEATHER

Unfortunately, we weren't able to find a data source that would cover the entire timespan and would at the same time have high spatial granularity. In the end, we opted for 'https://open-meteo.com/en/docs/historical-weather-api', which provides hourly weather data for the entire New York. From the available information we took the temperature and the weather code, and downloaded data for the entire required time period with one API call. We made sure to adjust for the timezone.

In [11]:
def load_weather_data(PATH):
    weather = dd.read_csv(PATH)
    
    # Handle datetime operations
    weather['time'] = dd.to_datetime(weather['time'])
    weather['time'] = weather['time'].dt.tz_localize('GMT').dt.tz_convert('America/New_York')
    weather['time'] = weather['time'].dt.tz_localize(None)
    
    # Convert columns to appropriate types
    weather['Temperature'] = weather['Temperature'].astype(float)
    weather['Weather Code'] = weather['Weather Code'].astype(int)
    
    return weather


In [12]:
WEATHER_PATH = 'data/weather_data.csv'
weather_ddf = load_weather_data(WEATHER_PATH)
weather_gdf = weather_ddf.compute()
print(weather_gdf.head())

                 time  Temperature  Weather Code
0 2007-12-31 19:00:00          1.6             0
1 2007-12-31 20:00:00          0.3             0
2 2007-12-31 21:00:00          0.3             0
3 2007-12-31 22:00:00         -0.3             0
4 2007-12-31 23:00:00         -0.6             0


## SCHOOLS

We obtained the school data from the following source: 'https://data.cityofnewyork.us/Education/2019-2020-School-Locations/wg9x-4ke6/data_preview'. Since vast majority of entries came equipped with exact coordinates, no much preprocessing was required. We kept the location and the name information and discarded the rest of the dataset.

In [13]:
def load_schools_data(PATH):
    dtypes = {
        'nta_name': 'str',
        'location_category_description': 'str',
        'latitude': 'float64',
        'longitude': 'float64'
    }
    schools = dd.read_csv(PATH, usecols=['nta_name', 'location_category_description', 'latitude', 'longitude'], dtype=dtypes)
    
    # Combine name columns
    schools['Name'] = schools['nta_name'].astype(str) + ' ' + schools['location_category_description'].astype(str)
    
    # Only keep relevant columns
    schools = schools[['Name', 'latitude', 'longitude']]
    
    # Create geometry column in Dask
    def make_point(row):
        return Point(row['longitude'], row['latitude'])

    # Use map_partitions for custom geometry creation
    schools['Location'] = schools.map_partitions(lambda df: df.apply(make_point, axis=1), meta=('Location', 'geometry'))
    
    # Convert to GeoDataFrame
    schools_gdf = dgpd.from_dask_dataframe(schools, geometry='Location')
    schools_gdf = schools_gdf.set_crs("EPSG:4326").to_crs(3857)
    
    # Drop unnecessary columns
    schools_gdf = schools_gdf.drop(columns=['longitude', 'latitude'])
    
    return schools_gdf

In [14]:
SCHOOLS_PATH = 'data/schools.csv'
school_ddf = load_schools_data(SCHOOLS_PATH)
school_gdf = school_ddf.compute()
print(school_gdf.head())

                                                Name  \
0                        Sunset Park West Elementary   
1  Prospect Lefferts Gardens-Wingate Junior High-...   
2                            Clinton Hill Elementary   
3                           East New York Elementary   
4                      Stuyvesant Heights Elementary   

                           Location  
0  POINT (-8238913.587 4960700.272)  
1   POINT (-8232251.672 4961795.46)  
2  POINT (-8232657.321 4965594.938)  
3  POINT (-8224203.384 4962100.238)  
4  POINT (-8228956.059 4966025.055)  


## EVENTS

Lastly, the event information was obtained from the following source: 'https://data.cityofnewyork.us/City-Government/NYC-Permitted-Event-Information-Historical/bkfu-528j/about_data'. Since this time, no information about exact location was included, we had to infer location based on the police precinct, responsible for the event. We found a mapping from police precincts to shape files here: 'https://data.cityofnewyork.us/City-Government/Police-Precincts/y76i-bdw7/about_data'. Once again, we calculated the centroids for each precinct, joined the two dataset on police precincts, and used this information as our best estimation of event location.

### Precincts

In [15]:
def load_precincts_data(PATH):
    precincts = dd.read_csv(PATH, usecols=['precinct', 'the_geom'], dtype={'precinct': 'int64', 'the_geom': 'object'})
    
    # Apply ast.literal_eval on the 'the_geom' column
    precincts['the_geom'] = precincts['the_geom'].apply(ast.literal_eval, meta=('x', 'object'))
    
    # Create geometry using map_partitions
    precincts['geometry'] = precincts.map_partitions(lambda df: df['the_geom'].apply(shape), meta=('geometry', 'geometry'))
    precincts = precincts.drop(columns=['the_geom'])
    
    # Create GeoDataFrame
    gdf = dgpd.from_dask_dataframe(precincts, geometry='geometry')
     
    # Calculate centroids
    gdf['centroid'] = gdf.geometry.centroid
    
    # Drop unnecessary columns
    gdf = gdf.drop(columns=['geometry'])
    
    # Ensure 'precinct' column is of integer type
    gdf['precinct'] = gdf['precinct'].astype(int)
    
    
    return gdf

In [16]:
PRECINCTS_PATH = 'data/police_precincts.csv'
precincts = load_precincts_data(PRECINCTS_PATH)
precincts = precincts.compute()
print(precincts.head())

   precinct                    centroid
0         1  POINT (-74.01207 40.70979)
1         5  POINT (-73.99715 40.71641)
2         6  POINT (-74.00238 40.73367)
3         7  POINT (-73.98395 40.71544)
4         9  POINT (-73.98339 40.72627)


### Preload evets

In [17]:
def load_events_data(PATH):
    events = dd.read_csv(PATH, usecols=['Event Name', 'Start Date/Time', 'End Date/Time', 'Police Precinct'], dtype={'Event Name': 'str', 'Start Date/Time': 'str', 'End Date/Time': 'str', 'Police Precinct': 'str'})
    # Columns selection and transformations
    events = events[['Event Name', 'Start Date/Time', 'End Date/Time', 'Police Precinct']]
    events['Police Precinct'] = events['Police Precinct'].apply(lambda x: str(x).split(',')[0], meta=('x', 'str'))
    events['Police Precinct'] = dd.to_numeric(events['Police Precinct'], errors='coerce')
    events['Police Precinct'] = events['Police Precinct'].astype('Int64')  # Use Int64 for nullable integers

    
    # Handle datetime
    events['Start Date/Time'] = dd.to_datetime(events['Start Date/Time'])
    events['End Date/Time'] = dd.to_datetime(events['End Date/Time'])
    
    return events
    

In [18]:
EVENTS_PATH = 'data/events_sample.csv'
events = load_events_data(EVENTS_PATH)
events = events.compute()
print(events.head())

                                         Event Name     Start Date/Time  \
0                                  Big Apple Circus 2017-11-18 19:00:00   
1                          Mt. Eden Farmer's Market 2017-11-16 08:00:00   
2                    Columbia  Greenmarket Thursday 2017-11-21 08:00:00   
3                                  Lawn Maintenance 2017-11-23 00:00:00   
4  October, November December model aircraft flying 2017-11-22 09:00:00   

        End Date/Time  Police Precinct  
0 2017-11-18 20:00:00               20  
1 2017-11-16 16:00:00               44  
2 2017-11-21 17:00:00               26  
3 2017-11-23 23:58:00               13  
4 2017-11-22 20:00:00              122  


### Merge Precincts and events

In [19]:
def merge_events_precincts(events, precincts):
    events['Police Precinct'] = events['Police Precinct'].astype('Int64')  # Ensure the type matches
    precincts['precinct'] = precincts['precinct'].astype('Int64')  # Ensure the type matches
    precincts = precincts.set_geometry('centroid')
    precincts = precincts.set_crs("EPSG:4326")
    merged = dd.merge(events, precincts, how='left', left_on='Police Precinct', right_on='precinct')
    merged = merged.drop(columns=['precinct', 'Police Precinct'])
    merged = merged.rename(columns={'centroid': 'Location'})
    merged = dgpd.from_dask_dataframe(merged, geometry='Location')
    merged = merged.set_crs("EPSG:4326").to_crs(3857)

    
    return merged

In [20]:
precincts = load_precincts_data(PRECINCTS_PATH)
events = load_events_data(EVENTS_PATH)
merged = merge_events_precincts(events, precincts)
merged = merged.compute()
print(merged.head())

                                         Event Name     Start Date/Time  \
0                                  Big Apple Circus 2017-11-18 19:00:00   
1                          Mt. Eden Farmer's Market 2017-11-16 08:00:00   
2                    Columbia  Greenmarket Thursday 2017-11-21 08:00:00   
3                                  Lawn Maintenance 2017-11-23 00:00:00   
4  October, November December model aircraft flying 2017-11-22 09:00:00   

        End Date/Time                          Location  
0 2017-11-18 20:00:00  POINT (-8235604.809 4979855.952)  
1 2017-11-16 16:00:00   POINT (-8228825.29 4987858.923)  
2 2017-11-21 17:00:00  POINT (-8233067.187 4984649.037)  
3 2017-11-23 23:58:00  POINT (-8235910.115 4973779.129)  
4 2017-11-22 20:00:00    POINT (-8250545.9 4949933.985)  


## JOIN WITH ORIGINAL DATASET

The hardest part of data enrichment task came in the form of joining. 

Firstly, in order to do any kind of spatial calculations, a regular dataframe had to be converted into a geo-dataframe with a designated geometry column. However, since each geo-dataframe can only have one single geometry column (our dataset would require 2 - one for pickup and one for dropoff location), we had to create two separate geodataframes. For the purposes of proving the concept, the rest of the calculations were only made on the dropoff location based geodataframe.

Join with the weather data was relatively straight forward. Timestamps from the original dataset were rounded to hours since that was the granularity of our weather information. After that, a simple join was performed.

Schools and businesses were a bit trickier. While in vanilla pandas, a simple *sjoin_nearest* could be performed, dask version of geopandas doesn't support that operation, since by default it can't be parallelized. In order to find the closest school/business, the entire dataset has to be loaded into memory. The problem was solved using the cKDTree algorithm, which uses a binary tree-like algorith, to find the closest neighbor effitiently. And while the business/school data had to be loaded into memory, the original dataset can be processed in batches.

Lastly, we had to join the events data. This task proved to be the most tedeous, since the join had to take into account both spatial and temporal conditions. We tackled the problem by first dealing with the spatial part. We first decided on an arbitrary vicinity condition (1km for example). We created a buffer around each taxi PU/DO location of that size and then performed a spatial join with the business dataset. However, since dask implementation currently only supports inner joins, another join with the original dataset was required in order to retain information about taxi trips, that did not end in a vicinity of an event. With that, we got a dataset containing one row for each trip-event combination, that were within the set distance, and one row for each trip that didn't end in a vicinity of an event. To enforce the temporal condition, we calculated a new time difference column, which recorder the time difference between event  and taxi trip. We then filtered the entire dataset based on another arbitrary temporal condition (1h for example). This was, we ended up with a dataset, the contained one row for each taxi trip-event combination, where the event started within 1km and 2hours of the taxi trip. 

In [22]:
from scipy.spatial import cKDTree
import numpy as np

"""  Read and prepare the data """

DATA_PATH = 'data/example.parquet'
df = dd.read_parquet(DATA_PATH, engine='pyarrow')

df = df.dropna(subset=['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'])
df['PU Location'] = df.apply(lambda r: Point(r['pickup_longitude'], r['pickup_latitude']), axis=1, meta=('PU Lucation', 'geometry'))
df['DO Location'] = df.apply(lambda r: Point(r['dropoff_longitude'], r['dropoff_latitude']), axis=1, meta=('DO Lucation', 'geometry'))

# pickup_gdf = dgpd.from_dask_dataframe(df, geometry='PU Location')
# pickup_gdf = pickup_gdf.drop(columns=['DO Location'])
# pickup_gdf = pickup_gdf.set_crs("EPSG:4326").to_crs(3857)

dataset = dgpd.from_dask_dataframe(df, geometry='DO Location')
dataset = dataset.drop(columns=['PU Location'])
dataset = dataset.set_crs("EPSG:4326").to_crs(3857)


""" Join the data with the weather data """

WEATHER_PATH = 'data/weather_data.csv'
weather_ddf = load_weather_data(WEATHER_PATH)

dataset['dropoff_datetime'] = dd.to_datetime(dataset['dropoff_datetime'])
dataset['dropoff_time_rounded'] = dataset['dropoff_datetime'].dt.round('h')

dataset = dataset.merge(weather_ddf, how='left', left_on='dropoff_time_rounded', right_on='time')
dataset = dataset.drop(columns=['dropoff_time_rounded', 'time'])

""" Join with the schools data - dask geo doesn't have sjoin-nearest (has to be computed in memory) """

SCHOOLS_PATH = 'data/schools.csv'
school_ddf = load_schools_data(SCHOOLS_PATH)

dropoff_gdf = dataset.compute()
schools_gdf = school_ddf.compute()

# Ensure CRS match
# dropoff_gdf = dropoff_gdf.to_crs(3857)
# schools_gdf = schools_gdf.to_crs(3857)

# Extract coordinate arrays
dropoff_coords = np.array(list(zip(dropoff_gdf.geometry.x, dropoff_gdf.geometry.y)))
school_coords = np.array(list(zip(schools_gdf.geometry.x, schools_gdf.geometry.y)))

dropoff_coords = dropoff_coords[~np.isnan(dropoff_coords).any(axis=1)]
school_coords = school_coords[~np.isnan(school_coords).any(axis=1)]

# Build KDTree and query nearest neighbors
tree = cKDTree(school_coords)
distances, indices = tree.query(dropoff_coords, k=1)

# Get nearest school info
nearest_schools = schools_gdf.iloc[indices].reset_index(drop=True)

# Add nearest info back to dropoff_gdf
dropoff_gdf['dist_from_school'] = distances
dropoff_gdf['nearest_school_name'] = nearest_schools['Name'].values


# dropoff_gdf = dd.from_pandas(dropoff_gdf)

""" Join with the business data - dask geo doesn't have sjoin-nearest (has to be computed in memory) """

BUSINESS_PATH = 'data/businesses.csv'
bsns = load_business_data(BUSINESS_PATH)
zip_df = load_zip_data(ZIP_PATH)
bsns_gdf = merge_business_zip(bsns, zip_df)
bsns_gdf = bsns_gdf.compute()

bsns_coords = np.array(list(zip(bsns_gdf.geometry.x, bsns_gdf.geometry.y)))
tree = cKDTree(bsns_coords)
distances, indices = tree.query(dropoff_coords, k=1)
nearest_bsns = bsns_gdf.iloc[indices].reset_index(drop=True)
dropoff_gdf['dist_from_business'] = distances
dropoff_gdf['nearest_business_name'] = nearest_bsns['Business Name'].values




""" Join with the events data  """

PRECINCTS_PATH = 'data/police_precincts.csv'
EVENTS_PATH = 'data/events_sample.csv'
precincts = load_precincts_data(PRECINCTS_PATH)
events_pre = load_events_data(EVENTS_PATH)
events = merge_events_precincts(events_pre, precincts)





# Start with your GeoDataFrames
dropoff_gdf = dropoff_gdf.reset_index(drop=True)
dropoff_gdf['row_id'] = dropoff_gdf.index


dropoff_buffered = dropoff_gdf.copy()
dropoff_buffered['DO Location'] = dropoff_buffered.buffer(100)
matches = dgpd.sjoin(dropoff_buffered, merged, predicate='intersects')


# Keep a copy of the geometry column
dropoff_geometry = dropoff_gdf.geometry
matches_geometry = matches.geometry

# Convert to Dask DataFrames (dropping geometry temporarily)
dropoff_df = dropoff_gdf.drop(columns='DO Location')
matches_df = matches.drop(columns='DO Location')


event_cols = [col for col in matches_df.columns 
              if col not in dropoff_df.columns or col == "row_id"]
matches_df = matches_df[event_cols]

# Merge as plain Dask DataFrames
joined = dd.merge(dropoff_df, matches_df, how='left', on='row_id')

# Reattach the original geometry (optional: you could pick one of them)
joined['DO Location'] = dropoff_geometry

# Convert back to Dask GeoDataFrame
joined_gdf = dgpd.from_dask_dataframe(joined, geometry='DO Location')


joined_gdf['time_diff'] = np.abs((joined_gdf['Start Date/Time'] - joined_gdf['dropoff_datetime']).dt.total_seconds() / 3600)
joined_gdf = joined_gdf[(joined_gdf['time_diff'] <= 1) | (joined_gdf['time_diff'].isna())]


joined_gdf = joined_gdf.drop(columns=['row_id', 'index_right', 'Start Date/Time', 'End Date/Time'])
display(joined_gdf.head())


,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,rate_code_id,store_and_fwd_flag,payment_type,fare_amount,extra,...,dropoff_latitude,Temperature,Weather Code,dist_from_school,nearest_school_name,dist_from_business,nearest_business_name,Event Name,DO Location,time_diff
0,2,2024-12-01 00:12:27,2024-12-01 00:31:12,1,9.76,1,0,1,38.000000,6.0,...,40.695797,-2.8,0,769.353141,Brooklyn Heights-Cobble Hill K-8,70.814723,PDH DESIGN AND CONSTRUCTION LLC,<NA>,POINT (-8237113.205 4967574.804),NaN
1,2,2024-11-30 23:56:04,2024-12-01 00:28:15,1,7.62,1,0,1,37.299999,1.0,...,40.818256,-2.5,0,435.805406,Central Harlem North-Polo Grounds Elementary,77.150995,2400 DELI & GROCERY CORPORATION,<NA>,POINT (-8231049.2 4985571.368),NaN
2,2,2024-12-01 00:50:35,2024-12-01 01:24:46,4,20.07,2,0,2,70.000000,0.0,...,40.780437,-2.8,0,166.127908,Upper East Side-Carnegie Hill K-12 all grades,101.501429,1065 PARK AVENUE GARAGE LLC,<NA>,POINT (-8232856.511 4980009.983),NaN
3,2,2024-12-01 00:18:16,2024-12-01 00:33:16,3,2.34,1,0,1,15.600000,1.0,...,40.748497,-2.8,0,261.694336,Midtown-Midtown South High school,19.407107,ADS CONSTRUCTION CORP.,<NA>,POINT (-8236800.662 4975315.508),NaN
4,2,2024-12-01 00:56:13,2024-12-01 01:18:25,1,5.05,1,0,1,26.799999,1.0,...,40.715370,-2.8,0,665.810341,Bushwick South High school,273.248151,T.B.I. LLC,<NA>,POINT (-8230605.866 4970449.047),NaN
